In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load CSV (replace with your actual file path)
df = pd.read_csv("/content/drive/MyDrive/AUV Implenation/combined_fixed_labels.csv")

# Replace 'label' with the actual column name of your labels
label_counts = df['label'].value_counts()

print(label_counts)


label
0    19700000
1    19700000
Name: count, dtype: int64


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA A100-SXM4-40GB


In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os

def create_balanced_sample(csv_path, output_path, sample_size=1000000, label_col='label', chunksize=100000):
    """
    Create a balanced sample from a large CSV file with equal 0s and 1s in the label column.

    Args:
        csv_path: Path to the input CSV file
        output_path: Path to save the balanced sample
        sample_size: Total number of rows in the balanced sample
        label_col: Name of the label column
        chunksize: Number of rows to read at a time
    """

    # Count total rows and label distribution first
    print("Counting rows and label distribution...")
    total_rows = 0
    label_counts = defaultdict(int)

    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        total_rows += len(chunk)
        chunk_labels = chunk[label_col].value_counts().to_dict()
        for label, count in chunk_labels.items():
            # Convert label to int for consistency
            if isinstance(label, str) and label.lower() == "normal":
                label_counts[0] += count
            else:
                label_counts[int(label)] += count

    print(f"Total rows: {total_rows:,}")
    print(f"Label distribution: {dict(label_counts)}")

    # Calculate how many of each label we need
    samples_per_label = sample_size // 2
    print(f"Need {samples_per_label:,} samples for each label")

    # Check if we have enough samples for each label
    for label, count in label_counts.items():
        if count < samples_per_label:
            print(f"Warning: Only {count:,} samples available for label {label}, but need {samples_per_label:,}")
            samples_per_label = min(samples_per_label, count)

    # Reset to actual sample size
    sample_size = samples_per_label * 2
    print(f"Final sample size: {sample_size:,}")

    # Initialize collections for each label
    sampled_data = {0: [], 1: []}

    # Read through file again and sample rows
    print("Sampling balanced data...")
    rows_processed = 0

    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        # Process each row in the chunk
        for idx, row in chunk.iterrows():
            # Get label value
            label_val = row[label_col]

            # Convert label to int (0 for "normal", 1 for others)
            if isinstance(label_val, str) and label_val.lower() == "normal":
                label_int = 0
            else:
                label_int = int(label_val)

            # Only consider labels 0 and 1
            if label_int not in [0, 1]:
                continue

            # Check if we need more samples for this label
            if len(sampled_data[label_int]) < samples_per_label:
                sampled_data[label_int].append(row)

            # Check if we have enough samples
            if (len(sampled_data[0]) >= samples_per_label and
                len(sampled_data[1]) >= samples_per_label):
                break

        rows_processed += len(chunk)
        print(f"Processed {rows_processed:,} rows. "
              f"Collected: 0s={len(sampled_data[0]):,}, 1s={len(sampled_data[1]):,}")

        # Break early if we have enough samples
        if (len(sampled_data[0]) >= samples_per_label and
            len(sampled_data[1]) >= samples_per_label):
            break

    # Combine the sampled data
    print("Combining samples...")
    sampled_rows = sampled_data[0] + sampled_data[1]

    # Convert to DataFrame
    sampled_df = pd.DataFrame(sampled_rows)

    # Shuffle the DataFrame
    sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Verify the balance
    label_dist = sampled_df[label_col].value_counts()
    print(f"Final label distribution:\n{label_dist}")

    # Save to CSV
    print(f"Saving to {output_path}...")
    sampled_df.to_csv(output_path, index=False)
    print("Done!")

    return sampled_df

# Usage
if __name__ == "__main__":
    input_csv = "/content/drive/MyDrive/AUV Implenation/combined_fixed_labels.csv"
    output_csv = "/content/drive/MyDrive/AUV Implenation/balanced_sample_1M.csv"

    # Create the balanced sample
    balanced_data = create_balanced_sample(input_csv, output_csv)

Counting rows and label distribution...
Total rows: 39,400,000
Label distribution: {1: 19700000, 0: 19700000}
Need 500,000 samples for each label
Final sample size: 1,000,000
Sampling balanced data...
Processed 100,000 rows. Collected: 0s=49,911, 1s=50,089
Processed 200,000 rows. Collected: 0s=100,000, 1s=100,000
Processed 300,000 rows. Collected: 0s=149,937, 1s=150,063
Processed 400,000 rows. Collected: 0s=200,000, 1s=200,000
Processed 500,000 rows. Collected: 0s=249,963, 1s=250,037
Processed 600,000 rows. Collected: 0s=300,000, 1s=300,000
Processed 700,000 rows. Collected: 0s=350,113, 1s=349,887
Processed 800,000 rows. Collected: 0s=400,000, 1s=400,000
Processed 900,000 rows. Collected: 0s=449,978, 1s=450,022
Processed 1,000,000 rows. Collected: 0s=500,000, 1s=500,000
Combining samples...
Final label distribution:
label
1    500000
0    500000
Name: count, dtype: int64
Saving to /content/drive/MyDrive/AUV Implenation/balanced_sample_1M.csv...
Done!


In [ ]:
import pandas as pd
import numpy as np

# ---------------- CONFIG ----------------
csv_path = '/content/drive/MyDrive/AUV Implenation/balanced_sample_1M.csv'  # update path
label_col = 'label'

# ---------------- LOAD DATA ----------------
df = pd.read_csv(csv_path)
print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns\n")

# ---------------- COLUMN TYPES ----------------
print("[INFO] Column data types:")
print(df.dtypes)
print()

# ---------------- MISSING VALUES ----------------
print("[INFO] Missing values per column:")
print(df.isna().sum())
print()

# ---------------- LABEL DISTRIBUTION ----------------
print("[INFO] Label distribution:")
if df[label_col].dtype == object:
    print(df[label_col].value_counts())
else:
    print(df[label_col].value_counts(normalize=True))
print()

# ---------------- NUMERIC FEATURE STATS ----------------
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if label_col in numeric_cols:
    numeric_cols.remove(label_col)

print(f"[INFO] Numeric feature stats ({len(numeric_cols)} columns):")
print(df[numeric_cols].describe())
print()

# ---------------- CATEGORICAL FEATURE CHECK ----------------
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if categorical_cols:
    print(f"[INFO] Categorical columns: {categorical_cols}")
    for col in categorical_cols:
        print(f"  {col} unique values: {df[col].nunique()}")
        print(f"  Sample values: {df[col].unique()[:10]}")
else:
    print("[INFO] No categorical columns detected.")

# ---------------- OPTIONAL: Check labels are integers 0/1 ----------------
if df[label_col].dtype != np.int64 and df[label_col].dtype != np.int32:
    print(f"\n[WARNING] Label column '{label_col}' is not integer type. Sample values:")
    print(df[label_col].unique()[:20])


[INFO] Dataset loaded: 1000000 rows, 7 columns

[INFO] Column data types:
Date                  object
Time (UTC)            object
Salinity (ppt)       float64
Temperature (°C)     float64
Depth (m)            float64
Sound speed (m/s)    float64
label                  int64
dtype: object

[INFO] Missing values per column:
Date                 0
Time (UTC)           0
Salinity (ppt)       0
Temperature (°C)     0
Depth (m)            0
Sound speed (m/s)    0
label                0
dtype: int64

[INFO] Label distribution:
label
1    0.5
0    0.5
Name: proportion, dtype: float64

[INFO] Numeric feature stats (4 columns):
       Salinity (ppt)  Temperature (°C)       Depth (m)  Sound speed (m/s)
count  1000000.000000    1000000.000000  1000000.000000     1000000.000000
mean        31.085999         11.339168       28.754971        1487.826492
std          8.572060          1.228593       20.222632          18.807726
min         -2.687974          7.767922       -7.035373        1434.3035

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

# ---------------- CONFIG ----------------
input_csv = '/content/drive/MyDrive/AUV Implenation/combined_fixed_labels.csv'
output_csv = '/content/drive/MyDrive/AUV Implenation/clean_balanced_10M.csv'
n_samples = 10_000_000  # total rows
label_col = 'label'

# ---------------- LOAD & CLEAN ----------------
chunksize = 500_000
pos_samples, neg_samples = [], []
pos_collected, neg_collected = 0, 0
target_per_class = n_samples // 2

for chunk in pd.read_csv(input_csv, chunksize=chunksize):
    # 1️⃣ Fix numeric negatives
    for col in ['Salinity (ppt)', 'Depth (m)']:
        chunk[col] = chunk[col].clip(lower=0)

    # 2️⃣ Keep only necessary columns
    chunk = chunk[['Date', 'Time (UTC)', 'Salinity (ppt)', 'Temperature (°C)', 'Depth (m)', 'Sound speed (m/s)', label_col]]

    # 3️⃣ Split by label
    pos_chunk = chunk[chunk[label_col] == 1]
    neg_chunk = chunk[chunk[label_col] == 0]

    if pos_collected < target_per_class and len(pos_chunk) > 0:
        take = min(target_per_class - pos_collected, len(pos_chunk))
        pos_samples.append(pos_chunk.sample(take, random_state=42))
        pos_collected += take

    if neg_collected < target_per_class and len(neg_chunk) > 0:
        take = min(target_per_class - neg_collected, len(neg_chunk))
        neg_samples.append(neg_chunk.sample(take, random_state=42))
        neg_collected += take

    if pos_collected >= target_per_class and neg_collected >= target_per_class:
        break

# ---------------- COMBINE & SHUFFLE ----------------
df_balanced = pd.concat(pos_samples + neg_samples).sample(frac=1, random_state=42).reset_index(drop=True)

# ---------------- ENCODE TIME ----------------
df_balanced['hour'] = df_balanced['Time (UTC)'].str[:2].astype(int)
df_balanced['minute'] = df_balanced['Time (UTC)'].str[3:5].astype(int)
df_balanced['time_sin'] = np.sin(2 * np.pi * (df_balanced['hour']*60 + df_balanced['minute']) / 1440)
df_balanced['time_cos'] = np.cos(2 * np.pi * (df_balanced['hour']*60 + df_balanced['minute']) / 1440)

# Drop original Date/Time columns
df_balanced.drop(['Date', 'Time (UTC)', 'hour', 'minute'], axis=1, inplace=True)

# ---------------- SAVE ----------------
df_balanced.to_csv(output_csv, index=False)
print(f"[INFO] Cleaned and balanced dataset saved: {output_csv}")
print(df_balanced.head())
print(df_balanced[label_col].value_counts())


[INFO] Cleaned and balanced dataset saved: /content/drive/MyDrive/AUV Implenation/clean_balanced_10M.csv
   Salinity (ppt)  Temperature (°C)  Depth (m)  Sound speed (m/s)  label  \
0       33.026302         10.598071  42.568523        1488.410034      1   
1       33.760000         12.189000  38.450000        1494.760000      0   
2       32.490000          9.900000   7.680000        1446.480000      0   
3       34.800799         10.013184  20.609443        1490.846722      1   
4       31.630000         10.988000  10.390000        1493.070000      0   

   time_sin  time_cos  
0  -0.97237 -0.233445  
1   0.95882 -0.284015  
2   0.95882 -0.284015  
3  -0.97237 -0.233445  
4   0.95502 -0.296542  
label
1    5000000
0    5000000
Name: count, dtype: int64


In [ ]:
!python "/content/drive/MyDrive/AUV Implenation/Deep Learning.py"


[INFO] Using device: cuda
[INFO] Dataset loaded: 10000000 rows
[INFO] Data moved to GPU: torch.Size([10000000, 6])

[INFO] Epoch 1/40
[INFO] Epoch 1 Train Loss=0.6564, Val Loss=0.6318, Train Acc=0.5659, Val Acc=0.5886

[INFO] Epoch 2/40
[INFO] Epoch 2 Train Loss=0.6321, Val Loss=0.6278, Train Acc=0.5890, Val Acc=0.5916

[INFO] Epoch 3/40
[INFO] Epoch 3 Train Loss=0.6254, Val Loss=0.6197, Train Acc=0.5998, Val Acc=0.6060

[INFO] Epoch 4/40
[INFO] Epoch 4 Train Loss=0.6196, Val Loss=0.6144, Train Acc=0.6071, Val Acc=0.6108

[INFO] Epoch 5/40
[INFO] Epoch 5 Train Loss=0.6146, Val Loss=0.6015, Train Acc=0.6126, Val Acc=0.6277

[INFO] Epoch 6/40
[INFO] Epoch 6 Train Loss=0.6081, Val Loss=0.5934, Train Acc=0.6203, Val Acc=0.6341

[INFO] Epoch 7/40
[INFO] Epoch 7 Train Loss=0.6029, Val Loss=0.5903, Train Acc=0.6259, Val Acc=0.6397

[INFO] Epoch 8/40
[INFO] Epoch 8 Train Loss=0.5988, Val Loss=0.5842, Train Acc=0.6302, Val Acc=0.6431

[INFO] Epoch 9/40
[INFO] Epoch 9 Train Loss=0.5954, Val Loss

In [ ]:
import torch
import torch.nn as nn

# Teacher model definition
class TeacherNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TeacherNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.layers(x)

# Example usage
input_dim = 100   # put your actual input dim
num_classes = 10  # put your actual num classes
teacher = TeacherNet(input_dim, num_classes)

print("Teacher architecture:")
print(teacher)


Teacher architecture:
TeacherNet(
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [ ]:
import torch
import torch.nn as nn

# Student model definition
class StudentNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(StudentNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Example usage
input_dim = 100   # put your actual input dim
num_classes = 10  # put your actual num classes
student = StudentNet(input_dim, num_classes)

print("Student architecture:")
print(student)


Student architecture:
StudentNet(
  (fc1): Linear(in_features=100, out_features=32, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


In [ ]:
import torch
import os

# ---------- Config ----------
teacher_path = "/content/drive/MyDrive/AUV Implenation/results_full_gpu/best_teacher.pth"
student_path = "/content/drive/MyDrive/AUV Implenation/final_model_gpu.pt"   # if exists, else skip

# ---------- Dummy Dataset Check ----------
def verify_dataset(dataset):
    first_x, first_y = dataset[0]
    input_dim = first_x.shape[0] if len(first_x.shape) == 1 else first_x.numel()
    num_classes = len(set([y.item() for _, y in dataset]))
    print(f"[DATASET] input_dim={input_dim}, num_classes={num_classes}")
    return input_dim, num_classes

# ---------- Model Check ----------
def check_checkpoint(path):
    if not os.path.exists(path):
        print(f"[MISSING] {path} not found.")
        return None
    ckpt = torch.load(path, map_location="cpu")
    print(f"[FOUND] {path} ✅")
    print(f"   Keys in checkpoint: {list(ckpt.keys())[:10]} ...")
    return ckpt

# Example usage:
# Replace `train_dataset` with your dataset object
# input_dim, num_classes = verify_dataset(train_dataset)

teacher_ckpt = check_checkpoint(teacher_path)
student_ckpt = check_checkpoint(student_path)


[FOUND] /content/drive/MyDrive/AUV Implenation/results_full_gpu/best_teacher.pth ✅
   Keys in checkpoint: ['layers.0.weight', 'layers.0.bias', 'layers.1.weight', 'layers.1.bias', 'layers.4.weight', 'layers.4.bias', 'layers.5.weight', 'layers.5.bias', 'layers.8.weight', 'layers.8.bias'] ...
[FOUND] /content/drive/MyDrive/AUV Implenation/final_model_gpu.pt ✅
   Keys in checkpoint: ['model_state_dict', 'features', 'classes', 'training_time', 'device', 'rounds'] ...


In [ ]:
import torch
import pandas as pd

# -------------------------------
# Load real dataset
# -------------------------------
DATA_PATH = "/content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv"   # replace with actual path
df = pd.read_csv(DATA_PATH)

print("[INFO] Loaded dataset:", DATA_PATH)
print("[INFO] Shape:", df.shape)
print("[INFO] Columns:", df.columns.tolist())

# -------------------------------
# Feature & Target Split
# -------------------------------
# Example: assuming last column is the label
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

print("[INFO] Features shape:", X.shape)
print("[INFO] Labels shape:", y.shape)

# -------------------------------
# Input dimension & Num classes
# -------------------------------
input_dim = X.shape[1]
num_classes = len(set(y))

print("[INFO] input_dim =", input_dim)
print("[INFO] num_classes =", num_classes)

# -------------------------------
# Convert to torch tensors
# -------------------------------
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.long)

print("[INFO] Torch tensors created")
print("    X_train:", X_train.shape)
print("    y_train:", y_train.shape)
print("    Classes:", torch.unique(y_train))


[INFO] Loaded dataset: /content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv
[INFO] Shape: (5225634, 5)
[INFO] Columns: ['Salinity (ppt)', 'Temperature (°C)', 'Depth (m)', 'Sound speed (m/s)', 'label']
[INFO] Features shape: (5225634, 4)
[INFO] Labels shape: (5225634,)
[INFO] input_dim = 4
[INFO] num_classes = 2
[INFO] Torch tensors created
    X_train: torch.Size([5225634, 4])
    y_train: torch.Size([5225634])
    Classes: tensor([0, 1])


In [ ]:
# =========================
# KD from 6D Teacher -> 4D Student (Colab-ready)
# =========================
import os
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# -------------------------------
# Config
# -------------------------------
DATA_PATH = "/content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv"
TEACHER_PATH = "/content/drive/MyDrive/AUV Implenation/results_full_gpu/best_teacher.pth"
STUDENT_SAVE_PATH = "/content/drive/MyDrive/AUV Implenation/student_kd.pth"
METRICS_CSV = "/content/drive/MyDrive/AUV Implenation/student_kd_metrics.csv"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8192               # adjust if you hit OOM
EPOCHS = 50
LR = 1e-3
WEIGHT_DECAY = 1e-4
ALPHA = 0.7                     # KD weight
TEMP = 2.0                      # KD temperature
VAL_FRAC = 0.2
SEED = 42
NUM_WORKERS = 0                 # 0 safest in Colab
PIN_MEMORY = True if DEVICE.type == "cuda" else False
PRINT_EVERY = 100

torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"[INFO] Device: {DEVICE}")

# -------------------------------
# Teacher & Student Architectures
# -------------------------------
class TeacherDeep(nn.Module):
    def __init__(self, input_dim: int, num_classes: int):
        super().__init__()
        hidden = [512, 256, 128, 64]
        layers = []
        prev = input_dim
        for h in hidden:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.LayerNorm(h))
            layers.append(nn.LeakyReLU())
            layers.append(nn.Dropout(0.2))
            prev = h
        self.layers = nn.Sequential(*layers)
        self.out = nn.Linear(prev, num_classes)

    def forward(self, x):
        return self.out(self.layers(x))

class StudentNet(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_classes: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes),
        )
    def forward(self, x):
        return self.net(x)

# -------------------------------
# KD Loss
# -------------------------------
def kd_loss_fn(student_logits, teacher_logits, targets, T=2.0, alpha=0.7):
    ce = F.cross_entropy(student_logits, targets)
    kl = F.kl_div(
        F.log_softmax(student_logits / T, dim=1),
        F.softmax(teacher_logits / T, dim=1),
        reduction="batchmean"
    ) * (T * T)
    return alpha * kl + (1.0 - alpha) * ce

# -------------------------------
# Load Dataset
# -------------------------------
print("[INFO] Loading dataset...")
df = pd.read_csv(DATA_PATH)
df = df.apply(pd.to_numeric, errors="coerce").dropna()

X = df.iloc[:, :-1].values.astype(np.float32)
y = df.iloc[:, -1].values.astype(np.int64)
num_classes = int(len(np.unique(y)))
input_dim = X.shape[1]
print(f"[INFO] Data shape X={X.shape}, y={y.shape}, input_dim={input_dim}, num_classes={num_classes}")

X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(
    X, y, test_size=VAL_FRAC, random_state=SEED, stratify=y
)

# Standardize
mean_ = X_train_np.mean(axis=0, keepdims=True)
std_  = X_train_np.std(axis=0, keepdims=True)
std_[std_ == 0.0] = 1.0
X_train_np = (X_train_np - mean_) / std_
X_val_np   = (X_val_np   - mean_) / std_

X_train = torch.tensor(X_train_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.long)
X_val   = torch.tensor(X_val_np, dtype=torch.float32)
y_val   = torch.tensor(y_val_np, dtype=torch.long)

train_loader = DataLoader(
    TensorDataset(X_train, y_train),
    batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
)
val_loader = DataLoader(
    TensorDataset(X_val, y_val),
    batch_size=BATCH_SIZE*2, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
)

# -------------------------------
# Teacher (6D input)
# -------------------------------
teacher_input_dim = 6
teacher = TeacherDeep(teacher_input_dim, num_classes).to(DEVICE)

print("[INFO] Loading teacher checkpoint...")
sd = torch.load(TEACHER_PATH, map_location="cpu")
if isinstance(sd, dict) and "state_dict" in sd:
    sd = sd["state_dict"]
teacher.load_state_dict(sd, strict=True)
teacher.eval()

# -------------------------------
# Student
# -------------------------------
student_hidden = 32
student = StudentNet(input_dim=input_dim, hidden_dim=student_hidden, num_classes=num_classes).to(DEVICE)
optimizer = torch.optim.AdamW(student.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))

# -------------------------------
# Eval helper
# -------------------------------
@torch.no_grad()
def evaluate(model, data_loader):
    model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    for xb, yb in data_loader:
        xb = xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)
        logits = model(xb)
        loss = F.cross_entropy(logits, yb, reduction="sum")
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)
        loss_sum += loss.item()
    acc = correct / max(1, total)
    return loss_sum / max(1, total), acc

# -------------------------------
# Pre-KD eval
# -------------------------------
pre_loss, pre_acc = evaluate(student, val_loader)
print(f"[PRE-KD] Val Loss={pre_loss:.4f}, Val Acc={pre_acc:.4f}")

# -------------------------------
# Train KD
# -------------------------------
def train_kd(epochs=EPOCHS):
    metrics = []
    for epoch in range(1, epochs+1):
        student.train()
        t0 = time.time()
        running = 0.0
        for i, (xb, yb) in enumerate(train_loader, 1):
            xb = xb.to(DEVICE, non_blocking=True)
            yb = yb.to(DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(DEVICE.type == "cuda")):
                s_logits = student(xb)
                # pad zeros for teacher
                pad = torch.zeros(xb.size(0), 2, device=xb.device, dtype=xb.dtype)
                xb_teacher = torch.cat([xb, pad], dim=1)
                with torch.no_grad():
                    t_logits = teacher(xb_teacher)
                loss = kd_loss_fn(s_logits, t_logits, yb, T=TEMP, alpha=ALPHA)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running += loss.item()
            if i % PRINT_EVERY == 0:
                print(f"[Epoch {epoch}] Batch {i}/{len(train_loader)} KD-Loss={running/PRINT_EVERY:.4f}")
                running = 0.0
        val_loss, val_acc = evaluate(student, val_loader)
        dt = time.time() - t0
        print(f"[Epoch {epoch}] Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}  (epoch time {dt:.1f}s)")
        metrics.append({"epoch": epoch, "val_loss": val_loss, "val_acc": val_acc})
    return metrics

metrics = train_kd(EPOCHS)

# -------------------------------
# Post-KD eval & save
# -------------------------------
post_loss, post_acc = evaluate(student, val_loader)
print(f"[POST-KD] Val Loss={post_loss:.4f}, Val Acc={post_acc:.4f}")

os.makedirs(os.path.dirname(STUDENT_SAVE_PATH), exist_ok=True)
torch.save({
    "state_dict": student.state_dict(),
    "input_dim": input_dim,
    "num_classes": num_classes,
    "standardize_mean": mean_,
    "standardize_std": std_,
    "kd": {"alpha": ALPHA, "temperature": TEMP, "epochs": EPOCHS},
}, STUDENT_SAVE_PATH)
print(f"[INFO] KD student saved -> {STUDENT_SAVE_PATH}")

rows = [{"phase":"pre_kd","val_loss":pre_loss,"val_acc":pre_acc}]
rows += [{"phase":f"epoch_{m['epoch']}","val_loss":m["val_loss"],"val_acc":m["val_acc"]} for m in metrics]
rows += [{"phase":"post_kd","val_loss":post_loss,"val_acc":post_acc}]
pd.DataFrame(rows).to_csv(METRICS_CSV, index=False)
print(f"[INFO] Metrics saved -> {METRICS_CSV}")


[INFO] Device: cuda
[INFO] Loading dataset...
[INFO] Data shape X=(5225634, 4), y=(5225634,), input_dim=4, num_classes=2
[INFO] Loading teacher checkpoint...


/tmp/ipython-input-4287465235.py:144: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))


[PRE-KD] Val Loss=0.7167, Val Acc=0.4580


/tmp/ipython-input-4287465235.py:184: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type == "cuda")):


[Epoch 1] Batch 100/511 KD-Loss=0.3819
[Epoch 1] Batch 200/511 KD-Loss=0.3117
[Epoch 1] Batch 300/511 KD-Loss=0.2832
[Epoch 1] Batch 400/511 KD-Loss=0.2694
[Epoch 1] Batch 500/511 KD-Loss=0.2612
[Epoch 1] Val Loss=0.6343, Val Acc=0.5755  (epoch time 65.6s)
[Epoch 2] Batch 100/511 KD-Loss=0.2557
[Epoch 2] Batch 200/511 KD-Loss=0.2523
[Epoch 2] Batch 300/511 KD-Loss=0.2496
[Epoch 2] Batch 400/511 KD-Loss=0.2481
[Epoch 2] Batch 500/511 KD-Loss=0.2459
[Epoch 2] Val Loss=0.6277, Val Acc=0.5834  (epoch time 65.3s)
[Epoch 3] Batch 100/511 KD-Loss=0.2445
[Epoch 3] Batch 200/511 KD-Loss=0.2429
[Epoch 3] Batch 300/511 KD-Loss=0.2420
[Epoch 3] Batch 400/511 KD-Loss=0.2406
[Epoch 3] Batch 500/511 KD-Loss=0.2389
[Epoch 3] Val Loss=0.6240, Val Acc=0.5829  (epoch time 65.3s)
[Epoch 4] Batch 100/511 KD-Loss=0.2377
[Epoch 4] Batch 200/511 KD-Loss=0.2370
[Epoch 4] Batch 300/511 KD-Loss=0.2356
[Epoch 4] Batch 400/511 KD-Loss=0.2350
[Epoch 4] Batch 500/511 KD-Loss=0.2345
[Epoch 4] Val Loss=0.6215, Val Acc

In [ ]:
# =========================
# Test Saved KD Student Model
# =========================
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# -------------------------------
# Paths
# -------------------------------
DATA_PATH = "/content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv"  # dataset to test
STUDENT_SAVE_PATH = "/content/drive/MyDrive/AUV Implenation/student_kd.pth"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8192

print(f"[INFO] Device: {DEVICE}")

# -------------------------------
# Student Architecture (must match training)
# -------------------------------
class StudentNet(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_classes: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes),
        )
    def forward(self, x):
        return self.net(x)

# -------------------------------
# Safe load for PyTorch 2.6+ (weights + metadata)
# -------------------------------
import torch
torch.serialization.add_safe_globals([__import__('numpy')._core.multiarray._reconstruct])

ckpt = torch.load(STUDENT_SAVE_PATH, map_location=DEVICE, weights_only=False)
input_dim = ckpt["input_dim"]
num_classes = ckpt["num_classes"]
mean_ = ckpt["standardize_mean"]
std_  = ckpt["standardize_std"]

# -------------------------------
# Build student model and load weights
# -------------------------------
student_hidden = 32
student = StudentNet(input_dim=input_dim, hidden_dim=student_hidden, num_classes=num_classes).to(DEVICE)
student.load_state_dict(ckpt["state_dict"])
student.eval()
print(f"[INFO] Student model loaded from {STUDENT_SAVE_PATH}")

# -------------------------------
# Load dataset & standardize
# -------------------------------
df = pd.read_csv(DATA_PATH)
df = df.apply(pd.to_numeric, errors="coerce").dropna()

X = df.iloc[:, :-1].values.astype(np.float32)
y = df.iloc[:, -1].values.astype(np.int64)

# Standardize using training mean/std from KD
X_std = (X - mean_) / std_

X_tensor = torch.tensor(X_std, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

data_loader = DataLoader(
    TensorDataset(X_tensor, y_tensor),
    batch_size=BATCH_SIZE, shuffle=False
)

# -------------------------------
# Evaluation helper
# -------------------------------
@torch.no_grad()
def evaluate(model, data_loader):
    model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    for xb, yb in data_loader:
        xb = xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)
        logits = model(xb)
        loss = F.cross_entropy(logits, yb, reduction="sum")
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)
        loss_sum += loss.item()
    acc = correct / max(1, total)
    return loss_sum / max(1, total), acc

# -------------------------------
# Evaluate
# -------------------------------
val_loss, val_acc = evaluate(student, data_loader)
print(f"[TEST] Dataset Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")


[INFO] Device: cuda
[INFO] Student model loaded from /content/drive/MyDrive/AUV Implenation/student_kd.pth
[TEST] Dataset Val Loss=0.5945, Val Acc=0.6290


In [ ]:
# =========================
# Test Saved Student Model (Old or KD)
# =========================
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# -------------------------------
# Paths
# -------------------------------
DATA_PATH = "/content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv"
STUDENT_SAVE_PATH = "/content/drive/MyDrive/AUV Implenation/final_model_gpu.pt"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8192
print(f"[INFO] Device: {DEVICE}")

# -------------------------------
# Student Architecture
# -------------------------------
class StudentNet(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_classes: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes),
        )
    def forward(self, x):
        return self.net(x)

# -------------------------------
# Load checkpoint safely
# -------------------------------
import torch
torch.serialization.add_safe_globals([__import__('numpy')._core.multiarray._reconstruct])

ckpt = torch.load(STUDENT_SAVE_PATH, map_location=DEVICE, weights_only=False)

# Determine how to extract state_dict
if isinstance(ckpt, dict):
    if "state_dict" in ckpt:
        state_dict = ckpt["state_dict"]
        input_dim = ckpt.get("input_dim", 4)
        num_classes = ckpt.get("num_classes", 2)
        mean_ = ckpt.get("standardize_mean", np.zeros((1, input_dim)))
        std_ = ckpt.get("standardize_std", np.ones((1, input_dim)))
    elif "model_state_dict" in ckpt:  # old-style checkpoint
        state_dict = ckpt["model_state_dict"]
        input_dim = ckpt.get("features", 4)  # fallback if not in ckpt
        num_classes = ckpt.get("classes", 2)
        mean_ = np.zeros((1, input_dim))
        std_ = np.ones((1, input_dim))
    else:
        # assume dict is just weights
        state_dict = ckpt
        input_dim = 4
        num_classes = 2
        mean_ = np.zeros((1, input_dim))
        std_ = np.ones((1, input_dim))
else:
    # weights-only tensor
    state_dict = ckpt
    input_dim = 4
    num_classes = 2
    mean_ = np.zeros((1, input_dim))
    std_ = np.ones((1, input_dim))

# -------------------------------
# Build model and load weights
# -------------------------------
student = StudentNet(input_dim=input_dim, hidden_dim=32, num_classes=num_classes).to(DEVICE)

# Filter state_dict to match StudentNet keys
filtered_dict = {k.replace("net.", ""): v for k, v in state_dict.items() if "net" in k or k in student.state_dict()}
student.load_state_dict(filtered_dict, strict=False)
student.eval()
print(f"[INFO] Student model loaded from {STUDENT_SAVE_PATH}")

# -------------------------------
# Load dataset & standardize
# -------------------------------
df = pd.read_csv(DATA_PATH)
df = df.apply(pd.to_numeric, errors="coerce").dropna()

X = df.iloc[:, :-1].values.astype(np.float32)
y = df.iloc[:, -1].values.astype(np.int64)
X_std = (X - mean_) / std_

X_tensor = torch.tensor(X_std, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

data_loader = DataLoader(
    TensorDataset(X_tensor, y_tensor),
    batch_size=BATCH_SIZE, shuffle=False
)

# -------------------------------
# Evaluation helper
# -------------------------------
@torch.no_grad()
def evaluate(model, data_loader):
    model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    for xb, yb in data_loader:
        xb = xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)
        logits = model(xb)
        loss = F.cross_entropy(logits, yb, reduction="sum")
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)
        loss_sum += loss.item()
    acc = correct / max(1, total)
    return loss_sum / max(1, total), acc

# -------------------------------
# Evaluate
# -------------------------------
val_loss, val_acc = evaluate(student, data_loader)
print(f"[TEST] Dataset Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")


[INFO] Device: cuda
[INFO] Student model loaded from /content/drive/MyDrive/AUV Implenation/final_model_gpu.pt
[TEST] Dataset Val Loss=70.3039, Val Acc=0.5000


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv")
print(df.iloc[:, -1].value_counts())


label
0    2612817
1    2612817
Name: count, dtype: int64


In [ ]:
#!/usr/bin/env python3
"""
Federated Learning + Simple Blockchain ledger demo (no Ray/Flower)

Fixed: robust student checkpoint loader, AMP usage, label checks, safer saving.
All errors resolved including Subset evaluation bug and deprecated warnings.
"""
import os
import time
import json
import hashlib
import math
from copy import deepcopy
from typing import List, Dict, Any

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, matthews_corrcoef
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# For safe checkpoint fallback
from torch.serialization import add_safe_globals
import zipfile

# ---------------------------
# USER CONFIGURE THESE PATHS
# ---------------------------
CSV_PATH = "/content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv"  # <-- change to your CSV path
STUDENT_MODEL_PATH = "/content/drive/MyDrive/MyDrive/AUV Implenation/student_kd.pth"  # or None
OUTPUT_DIR = "/content/drive/MyDrive/AUV Implenation/fl_blockchain_results"

# ---------------------------
# TRAINING / FL CONFIG
# ---------------------------
NUM_CLIENTS_REQUESTED = 3
BATCH_SIZE = 1024
NUM_WORKERS = 0
LOCAL_EPOCHS = 1
ROUNDS = 50
LR = 3e-4
WEIGHT_DECAY = 1e-4
SEED = 42
VALIDATION_SPLIT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", DEVICE)
if DEVICE.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------------------
# MODEL DEFINITION
# ---------------------------
class Net(nn.Module):
    def __init__(self, inp, out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(inp, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, out)
        )

    def forward(self, x):
        return self.net(x)

# ---------------------------
# Blockchain simple implementation
# ---------------------------
class Block:
    def __init__(self, index: int, timestamp: float, update_hash: str, prev_hash: str, meta: Dict[str, Any]):
        self.index = index
        self.timestamp = timestamp
        self.update_hash = update_hash
        self.prev_hash = prev_hash
        self.meta = meta
        self.hash = self.compute_hash()

    def compute_hash(self) -> str:
        payload = f"{self.index}|{self.timestamp}|{self.update_hash}|{self.prev_hash}|{json.dumps(self.meta, sort_keys=True)}"
        return hashlib.sha256(payload.encode()).hexdigest()

    def to_dict(self):
        return {
            "index": self.index,
            "timestamp": self.timestamp,
            "update_hash": self.update_hash,
            "prev_hash": self.prev_hash,
            "meta": self.meta,
            "hash": self.hash
        }

class SimpleBlockchain:
    def __init__(self):
        self.chain: List[Block] = []
        genesis = Block(0, time.time(), update_hash="0"*64, prev_hash="0"*64, meta={"note": "genesis"})
        self.chain.append(genesis)

    def add_block(self, update_bytes: bytes, meta: Dict[str, Any]) -> Block:
        update_hash = hashlib.sha256(update_bytes).hexdigest()
        prev_hash = self.chain[-1].hash
        block = Block(len(self.chain), time.time(), update_hash, prev_hash, meta)
        if self.verify_block(block, update_bytes):
            self.chain.append(block)
            return block
        else:
            raise ValueError("Block verification failed")

    def verify_block(self, block: Block, update_bytes: bytes) -> bool:
        recomputed = hashlib.sha256(update_bytes).hexdigest()
        ok = (recomputed == block.update_hash) and (block.prev_hash == self.chain[-1].hash)
        return ok

    def to_json(self):
        return [blk.to_dict() for blk in self.chain]

# ---------------------------
# Robust student checkpoint loader
# ---------------------------
def try_load_student_checkpoint(path, map_location=None, verbose=True):
    """
    Attempt to safely load a student checkpoint.
    Returns loaded object or None on failure.
    """
    if path is None or not os.path.isfile(path):
        if verbose:
            print(f"[LOAD] Student checkpoint not found at: {path}")
        return None

    # 1) Try weights_only=True (safe): this expects a state_dict mapping name->tensor
    try:
        obj = torch.load(path, map_location=map_location, weights_only=True)
        if verbose:
            print("[LOAD] Loaded checkpoint with weights_only=True.")
        return obj
    except Exception as e:
        if verbose:
            print(f"[LOAD] weights_only=True failed: {type(e).__name__}: {e}")

    # 2) Try weights_only=False but with safe allowlist for numpy reconstruct (trusted)
    try:
        # attempt to find numpy reconstruct function
        safe_fn = None
        try:
            # In many numpy builds it's np.core.multiarray._reconstruct
            safe_fn = getattr(getattr(np, "core"), "multiarray")._reconstruct
        except Exception:
            safe_fn = None

        if safe_fn is not None:
            if verbose:
                print("[LOAD] Attempting torch.load(..., weights_only=False) with safe_globals for numpy reconstruct.")
            with add_safe_globals([safe_fn]):
                obj = torch.load(path, map_location=map_location, weights_only=False)
            if verbose:
                print("[LOAD] Loaded checkpoint with safe_globals (weights_only=False).")
            return obj
        else:
            if verbose:
                print("[LOAD] Could not locate numpy reconstruct function; skipping safe-globals fallback.")
    except Exception as e:
        if verbose:
            print(f"[LOAD] weights_only=False with safe_globals failed: {type(e).__name__}: {e}")

    # 3) If zip archive, show contents (helpful for debugging)
    try:
        with zipfile.ZipFile(path, "r") as z:
            if verbose:
                print("[LOAD] Checkpoint appears to be archive. Contents:")
                for info in z.infolist():
                    print("   ", info.filename)
    except Exception:
        pass

    if verbose:
        print("[LOAD] Failed to load checkpoint safely. Will continue with random init.")
    return None

# ---------------------------
# Utility helpers
# ---------------------------
def load_and_preprocess(csv_path):
    print(f"[INFO] Loading CSV: {csv_path}")
    df = pd.read_csv(csv_path)
    # Use the integer labels directly
    y = df.iloc[:, -1].values
    X_df = df.iloc[:, :-1]

    categorical_cols = X_df.select_dtypes(include='object').columns.tolist()
    numeric_cols = X_df.select_dtypes(exclude='object').columns.tolist()

    if len(categorical_cols) > 0:
        ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
        X_cat = ohe.fit_transform(X_df[categorical_cols])
    else:
        X_cat = np.empty((len(X_df), 0))

    if len(numeric_cols) > 0:
        scaler = StandardScaler()
        X_num = scaler.fit_transform(X_df[numeric_cols])
    else:
        X_num = np.empty((len(X_df), 0))

    X = np.hstack([X_num, X_cat]).astype(np.float32)
    unique, counts = np.unique(y, return_counts=True)
    print(f"[INFO] Data: samples={X.shape[0]} | features={X.shape[1]} | class_counts={dict(zip(unique.tolist(), counts.tolist()))}")
    if len(unique) < 2:
        raise RuntimeError("Dataset contains only a single class after mapping. Fix labels or CSV.")
    return X, y.astype(int)

def model_state_to_vector(state_dict: Dict[str, torch.Tensor]) -> np.ndarray:
    parts = []
    for k, v in state_dict.items():
        arr = v.detach().cpu().numpy().ravel()
        parts.append(arr)
    if len(parts) == 0:
        return np.array([], dtype=np.float32)
    return np.concatenate(parts).astype(np.float32)

def vector_to_state_dict_example(vector: np.ndarray, template_state: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    new_state = {}
    pos = 0
    for k, v in template_state.items():
        size = v.numel()
        chunk = vector[pos:pos+size].reshape(v.shape)
        new_state[k] = torch.tensor(chunk, dtype=v.dtype)
        pos += size
    return new_state

def subtract_state_dicts(a: Dict[str, torch.Tensor], b: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    out = {}
    for k in a.keys():
        out[k] = a[k].detach().cpu() - b[k].detach().cpu()
    return out

def add_state_dicts_weighted(base: Dict[str, torch.Tensor], updates: List[Dict[str, torch.Tensor]], weights: List[float]) -> Dict[str, torch.Tensor]:
    res = {}
    for k in base.keys():
        tensor = base[k].detach().cpu().clone()
        if tensor.dtype == torch.float32 or tensor.dtype == torch.float64:
            for upd, w in zip(updates, weights):
                tensor += (upd[k].detach().cpu() * w)
        else:
            # For non-float tensors (like BatchNorm running stats), just average
            for upd, w in zip(updates, weights):
                 tensor += upd[k].detach().cpu()
            tensor = tensor / len(updates) # Simple average

        res[k] = tensor
    return res

def state_dict_to_numpy_bytes(sd: Dict[str, torch.Tensor]) -> bytes:
    vec = model_state_to_vector(sd)
    return vec.tobytes()

# ---------------------------
# Local client training (sequential simulation)
# ---------------------------
def client_local_train(global_state: Dict[str, torch.Tensor],
                       dataset: TensorDataset,
                       indices: List[int],
                       client_id: int,
                       local_epochs: int = 1,
                       batch_size: int = 1024) -> Dict[str, torch.Tensor]:
    features = dataset.tensors[0].shape[1]
    classes = int(dataset.tensors[1].max().item()) + 1
    model = Net(features, classes).to(DEVICE)
    model.load_state_dict({k: v.to(DEVICE) for k, v in global_state.items()})

    sub = Subset(dataset, indices)
    loader = DataLoader(sub, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)

    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    loss_fn = nn.CrossEntropyLoss()
    # Fixed: Use updated torch.amp API
    scaler = torch.amp.GradScaler('cuda', enabled=(DEVICE.type == "cuda"))

    model.train()
    for ep in range(local_epochs):
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            # Fixed: Use updated torch.amp API
            with torch.amp.autocast('cuda', enabled=(DEVICE.type == "cuda")):
                logits = model(xb)
                loss = loss_fn(logits, yb)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

    new_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
    return new_state

# ---------------------------
# Evaluation helper - FIXED
# ---------------------------
def evaluate_state(state: Dict[str, torch.Tensor], dataset, batch_size: int = 2048):
    """
    Fixed evaluation function that properly handles both TensorDataset and Subset objects.
    """
    # Extract data from dataset - handle both TensorDataset and Subset
    if isinstance(dataset, TensorDataset):
        X_eval = dataset.tensors[0]
        y_eval = dataset.tensors[1]
    elif isinstance(dataset, Subset):
        # Fixed: Properly extract tensors from Subset
        indices = dataset.indices
        original_dataset = dataset.dataset  # This should be TensorDataset
        X_eval = original_dataset.tensors[0][indices]
        y_eval = original_dataset.tensors[1][indices]
    else:
        raise TypeError(f"Unsupported dataset type for evaluation: {type(dataset)}")

    # Handle empty dataset case
    if y_eval.numel() == 0:
        print("[WARN] Evaluation dataset is empty.")
        return {
            'accuracy': 0.0, 'precision_macro': 0.0, 'recall_macro': 0.0,
            'f1_macro': 0.0, 'mcc': None, 'roc_auc': None,
            'confusion_matrix': [[0, 0], [0, 0]]
        }

    features = X_eval.shape[1]
    classes = int(y_eval.max().item()) + 1

    model = Net(features, classes).to(DEVICE)
    model.load_state_dict({k: v.to(DEVICE) for k, v in state.items()})
    model.eval()

    loader = DataLoader(TensorDataset(X_eval, y_eval), batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)
    y_true, y_pred, y_scores = [], [], []

    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            probs = F.softmax(logits, dim=1)
            preds = probs.argmax(dim=1)
            y_true.extend(yb.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_scores.extend(probs.cpu().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_scores = np.array(y_scores)

    metrics = {}
    metrics['accuracy'] = float(accuracy_score(y_true, y_pred))
    metrics['precision_macro'] = float(precision_score(y_true, y_pred, average='macro', zero_division=0))
    metrics['recall_macro'] = float(recall_score(y_true, y_pred, average='macro', zero_division=0))
    metrics['f1_macro'] = float(f1_score(y_true, y_pred, average='macro', zero_division=0))
    metrics['mcc'] = float(matthews_corrcoef(y_true, y_pred)) if len(np.unique(y_true))>1 else None

    try:
        metrics['roc_auc'] = float(roc_auc_score(y_true, y_scores[:,1])) if y_scores.shape[1] > 1 else None
    except Exception:
        metrics['roc_auc'] = None

    metrics['confusion_matrix'] = confusion_matrix(y_true, y_pred).tolist()
    return metrics

# ---------------------------
# Main federated loop
# ---------------------------
def run_federated_blockchain(X, y):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if DEVICE.type == "cuda":
        torch.cuda.manual_seed_all(SEED)

    X_t = torch.tensor(X, dtype=torch.float32)
    y_t = torch.tensor(y, dtype=torch.long)
    dataset = TensorDataset(X_t, y_t)

    idxs = np.arange(len(dataset))
    train_idxs, val_idxs = train_test_split(idxs, test_size=VALIDATION_SPLIT, random_state=SEED, stratify=y)
    val_dataset = Subset(dataset, val_idxs)

    requested = max(1, NUM_CLIENTS_REQUESTED)
    client_splits = np.array_split(train_idxs, requested)
    client_splits = [arr.tolist() for arr in client_splits if len(arr)>0]
    NUM_CLIENTS = len(client_splits)
    print(f"[INFO] NUM_CLIENTS actual: {NUM_CLIENTS}")

    features = X.shape[1]
    classes = int(y.max()) + 1
    global_model = Net(features, classes).to(DEVICE)

    # Try to load student model robustly
    if STUDENT_MODEL_PATH is not None and os.path.isfile(STUDENT_MODEL_PATH):
        print("[INFO] Attempting to load student model from:", STUDENT_MODEL_PATH)
        loaded = try_load_student_checkpoint(STUDENT_MODEL_PATH, map_location=DEVICE, verbose=True)
        if loaded is None:
            print("[WARN] Student model could not be loaded safely; using random initialization.")
        else:
            # try to detect format
            try:
                if isinstance(loaded, dict) and all(isinstance(v, torch.Tensor) for v in loaded.values()):
                    global_model.load_state_dict(loaded)
                    print("[INFO] Loaded student state_dict into model.")
                elif isinstance(loaded, dict) and "model_state" in loaded:
                    global_model.load_state_dict(loaded["model_state"])
                    print("[INFO] Loaded checkpoint['model_state'] into model.")
                elif isinstance(loaded, dict) and "state_dict" in loaded:
                    global_model.load_state_dict(loaded["state_dict"])
                    print("[INFO] Loaded checkpoint['state_dict'] into model.")
                else:
                    # try to see if it's a mapping of numpy arrays -> convert
                    sd_try = {}
                    for k, v in loaded.items():
                        try:
                            sd_try[k] = torch.tensor(v)
                        except Exception:
                            sd_try = None
                            break
                    if sd_try is not None:
                        global_model.load_state_dict(sd_try)
                        print("[INFO] Loaded converted numpy->tensor state into model.")
                    else:
                        print("[WARN] Loaded checkpoint format not recognized. Starting from random init.")
            except Exception as e:
                print("[WARN] Failed to load checkpoint into model:", e)
                print("Continuing with random init.")

    global_state = {k: v.detach().cpu().clone() for k, v in global_model.state_dict().items()}

    ledger = SimpleBlockchain()
    ledger_debug_log = []
    rounds_history = []

    for rnd in range(1, ROUNDS+1):
        print(f"\n--- ROUND {rnd:03d} ---")
        start_round = time.time()
        client_updates = []
        client_num_samples = []

        for cid, c_idxs in enumerate(client_splits):
            print(f"[Client {cid}] Training on {len(c_idxs)} samples (round {rnd})")
            new_state = client_local_train(global_state, dataset, c_idxs, client_id=cid, local_epochs=LOCAL_EPOCHS, batch_size=BATCH_SIZE)
            delta = subtract_state_dicts(new_state, global_state)
            update_bytes = state_dict_to_numpy_bytes(delta)
            meta = {"client_id": cid, "round": rnd, "num_samples": len(c_idxs)}
            try:
                block = ledger.add_block(update_bytes, meta)
                ledger_debug_log.append({"action":"add_block", "round": rnd, "client": cid, "block_index": block.index, "block_hash": block.hash})
                print(f"[Blockchain] Block added for client {cid} -> block_index={block.index} hash={block.hash[:12]}...")
                client_updates.append(delta)
                client_num_samples.append(len(c_idxs))
            except Exception as e:
                print(f"[Blockchain] Block verification failed for client {cid}: {e}")
                ledger_debug_log.append({"action":"failed_block", "client": cid, "err": str(e)})

        if len(client_updates) == 0:
            print("No verified updates this round. Skipping aggregation.")
            continue

        total_samples = sum(client_num_samples)
        weights = [n / total_samples for n in client_num_samples]
        new_global_state = add_state_dicts_weighted(global_state, client_updates, weights)
        global_state = new_global_state

        round_time = time.time() - start_round
        val_metrics = evaluate_state(global_state, val_dataset, batch_size=BATCH_SIZE)
        print(f"[Round {rnd}] val_acc={val_metrics['accuracy']:.4f} val_f1={val_metrics['f1_macro']:.4f} time={round_time:.2f}s")

        rounds_history.append({
            "round": rnd,
            "val_accuracy": val_metrics['accuracy'],
            "val_f1_macro": val_metrics['f1_macro'],
            "val_precision_macro": val_metrics['precision_macro'],
            "val_recall_macro": val_metrics['recall_macro'],
            "val_roc_auc": val_metrics['roc_auc'],
            "num_verified_updates": len(client_updates),
            "round_time_s": round_time
        })

        if rnd % 5 == 0 or rnd == ROUNDS:
            torch.save(global_state, os.path.join(OUTPUT_DIR, f"global_state_round{rnd}.pth"))
            with open(os.path.join(OUTPUT_DIR, "ledger.json"), "w") as f:
                json.dump(ledger.to_json(), f, indent=2)

    final_model_path = os.path.join(OUTPUT_DIR, "final_model.pth")
    torch.save(global_state, final_model_path)
    print("[INFO] Final global model saved to", final_model_path)

    with open(os.path.join(OUTPUT_DIR, "ledger.json"), "w") as f:
        json.dump(ledger.to_json(), f, indent=2)
    with open(os.path.join(OUTPUT_DIR, "ledger_debug.json"), "w") as f:
        json.dump(ledger_debug_log, f, indent=2)

    df_hist = pd.DataFrame(rounds_history)
    metrics_csv = os.path.join(OUTPUT_DIR, "federated_metrics_rounds.csv")
    df_hist.to_csv(metrics_csv, index=False)
    print("[INFO] Metrics saved to:", metrics_csv)

    # plots
    try:
        plt.figure()
        plt.plot(df_hist["round"], df_hist["val_accuracy"], label="val_accuracy")
        plt.xlabel("Round"); plt.ylabel("Validation accuracy"); plt.title("Validation Accuracy per Round")
        plt.grid(True); plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, "val_accuracy.png")); plt.close()

        plt.figure()
        plt.plot(df_hist["round"], df_hist["val_f1_macro"], label="val_f1_macro")
        plt.xlabel("Round"); plt.ylabel("Validation F1 (macro)"); plt.title("Validation F1 per Round")
        plt.grid(True); plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, "val_f1.png")); plt.close()
        print("[INFO] Plots saved to", OUTPUT_DIR)
    except Exception as e:
        print("[WARN] Plotting failed:", e)

    return final_model_path, metrics_csv, os.path.join(OUTPUT_DIR, "ledger.json")

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    start = time.time()
    X, y = load_and_preprocess(CSV_PATH)
    final_model_path, metrics_csv, ledger_path = run_federated_blockchain(X, y)
    tot = time.time() - start
    print(f"\nALL DONE in {tot/60:.2f} minutes. Final model: {final_model_path}, metrics: {metrics_csv}, ledger: {ledger_path}")


Using device: cuda
GPU: NVIDIA L4
[INFO] Loading CSV: /content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv
[INFO] Data: samples=5225634 | features=4 | class_counts={0: 2612817, 1: 2612817}
[INFO] NUM_CLIENTS actual: 3

--- ROUND 001 ---
[Client 0] Training on 1567690 samples (round 1)
[Blockchain] Block added for client 0 -> block_index=1 hash=fee640dbc2ca...
[Client 1] Training on 1567690 samples (round 1)
[Blockchain] Block added for client 1 -> block_index=2 hash=a7390dbe5c0b...
[Client 2] Training on 1567690 samples (round 1)
[Blockchain] Block added for client 2 -> block_index=3 hash=4db18e9dec39...
[Round 1] val_acc=0.6147 val_f1=0.5749 time=61.86s

--- ROUND 002 ---
[Client 0] Training on 1567690 samples (round 2)
[Blockchain] Block added for client 0 -> block_index=4 hash=ba32aaa16323...
[Client 1] Training on 1567690 samples (round 2)
[Blockchain] Block added for client 1 -> block_index=5 hash=b8995d36695f...
[Client 2] Training on 1567690 samples (round 2)
[Blockc

In [ ]:
!pip install flwr pandas numpy scikit-learn matplotlib seaborn


In [ ]:
!pip install -U "flwr[simulation]"


In [ ]:
!pip uninstall -y flwr ray
!pip install -U "flwr[simulation]"


Found existing installation: flwr 1.20.0
Uninstalling flwr-1.20.0:
  Successfully uninstalled flwr-1.20.0
Found existing installation: ray 2.31.0
Uninstalling ray-2.31.0:
  Successfully uninstalled ray-2.31.0
  Using cached flwr-1.20.0-py3-none-any.whl.metadata (15 kB)
  Using cached ray-2.31.0-cp312-cp312-manylinux2014_x86_64.whl.metadata (13 kB)
Using cached ray-2.31.0-cp312-cp312-manylinux2014_x86_64.whl (66.7 MB)
Using cached flwr-1.20.0-py3-none-any.whl (617 kB)


In [ ]:
import ray
print(ray.__version__)


2.31.0


In [ ]:
import ray
print("Ray version:", ray.__version__)
import flwr
print("Flower version:", flwr.__version__)


Ray version: 2.31.0
Flower version: 1.20.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
#!/usr/bin/env python3
import os
import time
import gc
import pandas as pd
import numpy as np
import torch
import torch.cuda as cuda
import flwr as fl
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.metrics import precision_recall_fscore_support
from typing import Dict
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

warnings.filterwarnings('ignore')

# ----------------------------
# GOOGLE COLAB / MULTI-CLIENT CONFIG
# ----------------------------
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

# ----------------------------
# CONFIG
# ----------------------------
CSV_PATH = "/content/drive/MyDrive/AUV Implenation/balanced_sample_200MB.csv"
NUM_CLIENTS_REQUESTED = 4      # simulate clients
BATCH_SIZE = 512                 # reduce per-client batch size
NUM_WORKERS = 2
SEED = 42
NUM_ROUNDS = 20                  # reduce for testing on Colab
LEARNING_RATE = 0.001
PIN_MEMORY = True
SAVE_DIR = "/content/drive/MyDrive/AUV Implenation"

os.makedirs(SAVE_DIR, exist_ok=True)

# ----------------------------
# DEVICE SETUP
# ----------------------------
if not torch.cuda.is_available():
    raise RuntimeError("CUDA not available!")

DEVICE = torch.device("cuda:0")
print(f"Using device: {DEVICE}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
props = torch.cuda.get_device_properties(0)
print(f"GPU Memory: {props.total_memory / 1e9:.2f} GB")

torch.cuda.set_per_process_memory_fraction(0.85)

# ----------------------------
# METRICS TRACKER
# ----------------------------
class MetricsTracker:
    def __init__(self):
        self.client_metrics = defaultdict(list)
        self.global_metrics = []
        self.round_times = []

    def add_client_metrics(self, client_id: int, round_num: int, metrics: Dict):
        entry = {'client_id': client_id, 'round': round_num, **metrics}
        self.client_metrics[client_id].append(entry)

    def add_global_metrics(self, round_num: int, metrics: Dict):
        entry = {'round': round_num, **metrics}
        self.global_metrics.append(entry)

    def save_all(self, save_dir: str):
        if self.client_metrics:
            pd.concat([pd.DataFrame(v) for v in self.client_metrics.values()]).to_csv(
                f"{save_dir}/client_metrics.csv", index=False
            )
        if self.global_metrics:
            pd.DataFrame(self.global_metrics).to_csv(f"{save_dir}/global_metrics.csv", index=False)
        print(f"[METRICS] Saved metrics to {save_dir}")

metrics_tracker = MetricsTracker()

# ----------------------------
# LOAD DATA (once)
# ----------------------------
print("[INFO] Loading dataset...")
df = pd.read_csv(CSV_PATH, low_memory=False)
df = df.drop(columns=["Date", "Time (UTC)"], errors="ignore")
df = df.apply(pd.to_numeric, errors="coerce").dropna()

X = df.iloc[:, :-1].values.astype(np.float32)
y = df.iloc[:, -1].values.astype(np.int64)
dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
total_len = len(dataset)
print(f"[INFO] Samples: {total_len}")

# free temporary arrays
del X, y, df
gc.collect()

# ----------------------------
# CLIENT SPLIT (once) + PREPARE LOADERS ONCE
# ----------------------------
all_indices = np.arange(total_len)
np.random.seed(SEED)
np.random.shuffle(all_indices)

client_splits = np.array_split(all_indices, NUM_CLIENTS_REQUESTED)
client_indices_list = [arr.tolist() for arr in client_splits if arr.size > 0]
NUM_CLIENTS = len(client_indices_list)
print(f"[INFO] Using {NUM_CLIENTS} clients")

# Pre-create train/test loaders for each client (so we don't rebuild them every round)
client_train_loaders = []
client_test_loaders = []

for idxs in client_indices_list:
    n = len(idxs)
    train_size = max(1, int(0.8 * n))
    # deterministic split using the shuffled indices order
    train_idxs = idxs[:train_size]
    test_idxs = idxs[train_size:] if train_size < n else []

    train_subset = Subset(dataset, train_idxs)
    if test_idxs:
        test_subset = Subset(dataset, test_idxs)
    else:
        # if no test split (very small partition), reuse a tiny slice from train to avoid empty test set
        # (keeps behavior safe)
        test_subset = Subset(dataset, train_idxs[:1])

    train_loader = DataLoader(
        train_subset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS
    )
    test_loader = DataLoader(
        test_subset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS
    )

    client_train_loaders.append(train_loader)
    client_test_loaders.append(test_loader)

# ----------------------------
# MODEL
# ----------------------------
class OptimizedNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        return self.fc3(x)

# ----------------------------
# CLIENT CLASS
# ----------------------------
class EnhancedFlowerClient(fl.client.NumPyClient):
    def __init__(self, model, trainloader, testloader, cid):
        self.model = model.to(DEVICE)
        self.trainloader, self.testloader = trainloader, testloader
        self.cid = cid
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=LEARNING_RATE)

    def get_parameters(self, config=None):
        return [v.cpu().numpy() for v in self.model.state_dict().values()]

    def set_parameters(self, params):
        state_dict = {k: torch.tensor(v, device=DEVICE) for k, v in zip(self.model.state_dict().keys(), params)}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, params, config):
        self.set_parameters(params)
        self.model.train()
        for data, target in self.trainloader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            self.optimizer.zero_grad()
            loss = self.criterion(self.model(data), target)
            loss.backward()
            self.optimizer.step()
        return self.get_parameters(), len(self.trainloader.dataset), {}

    def evaluate(self, params, config):
        self.set_parameters(params)
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for data, target in self.testloader:
                data, target = data.to(DEVICE), target.to(DEVICE)
                pred = self.model(data).argmax(dim=1)
                correct += (pred == target).sum().item()
                total += target.size(0)
        return float(0.0), total, {"accuracy": correct / total}

# ----------------------------
# CLIENT FACTORY (uses pre-created loaders)
# ----------------------------
def create_client(cid):
    # Reuse the loaders created once above to avoid reloading/duplicating the dataset
    train_loader = client_train_loaders[cid]
    test_loader = client_test_loaders[cid]
    input_dim = dataset[0][0].shape[0]
    num_classes = len(torch.unique(dataset[:][1]))
    return EnhancedFlowerClient(OptimizedNet(input_dim, num_classes), train_loader, test_loader, cid)

def client_fn(context: fl.common.Context):
    cid = int(context.node_id) % NUM_CLIENTS
    return create_client(cid).to_client()

# ----------------------------
# TRAINING
# ----------------------------
def run_federated_training():
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=NUM_CLIENTS,
        min_evaluate_clients=NUM_CLIENTS,
        min_available_clients=NUM_CLIENTS,
    )
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
        strategy=strategy,
        client_resources={"num_cpus": 1, "num_gpus": 0.01},  # tiny GPU fraction per client
    )

if __name__ == "__main__":
    run_federated_training()


Using device: cuda:0
GPU: NVIDIA A100-SXM4-40GB
GPU Memory: 42.47 GB
[INFO] Loading dataset...
[INFO] Samples: 5225634


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout


[INFO] Using 4 clients


2025-08-26 10:21:36,675	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:A100': 1.0, 'node:__internal_head__': 1.0, 'CPU': 12.0, 'object_store_memory': 26799219916.0, 'node:172.28.0.12': 1.0, 'memory': 53598439835.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.01}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 12 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=23012) 2025-08-26 10:21:39.520910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=23012) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(p